<a href="https://colab.research.google.com/github/Kzis/mrt-is/blob/master/Prep_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [69]:
# df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv" , nrows=10000)
df = pd.read_csv("gdrive/My Drive/NIDA/transection_.csv")

In [70]:
df.shape

(17240769, 24)

In [106]:
df.TicketType.value_counts()

Card     11238083
Token     6002686
Name: TicketType, dtype: int64

In [107]:
df.TicketName.value_counts()

Adult-A      8945627
Adult        5811954
Student-A    1296077
Senior-A      520535
Adult-B       381360
Senior        161925
Student-B      52314
Child          28807
Senior-B       22109
Child-A        19229
Child-B          832
Name: TicketName, dtype: int64

In [109]:
df.CardType.value_counts()

1     8945627
58    5811954
4     1296077
3      520535
21     381360
60     161925
24      52314
59      28807
23      22109
2       19229
22        832
Name: CardType, dtype: int64

In [108]:
df.PassengerType.value_counts()

Adult      15138941
Student     1348391
Senior       704569
Child         48868
Name: PassengerType, dtype: int64

In [71]:
df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


In [72]:
def get_data_by_month(month):
  return df.loc[df['ThaiMonthName'] == month]

def get_transaction_df(month_df):
  df_month_transaction_date_sum = month_df.TransactionDate.value_counts().rename_axis('TransactionDate').reset_index(name='TransactionCounts')
  df_month_transaction_date_sum.sort_values('TransactionDate')
  return df_month_transaction_date_sum

In [73]:
def get_entry_total_station_by_date(month_df , month_transaction_df , col_name):
  date_list = month_transaction_df['TransactionDate'].tolist()
  col_list = month_df[col_name].unique().tolist()
  output = []
  for date in date_list:
    # print("Process : {}".format(date))
    for col_val in col_list:
      lst = []
      subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
      column_count = subset_df.value_counts()
      lst.extend([date,"Entry", col_val, column_count.index[0]])
      output.append(lst)
      # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
  # print(output)
  df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
  return df  

def get_exit_total_station_by_date(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    output = []
    for date in date_list:
      # print("Process : {}".format(date))
      for col_val in col_list:
        lst = []
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.extend([date,"Exit", col_val, column_count.index[0]])
        output.append(lst)
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
    # print(output)
    df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    return df  

In [74]:
jan_df = get_data_by_month("มกราคม")
jan_df.head()

,TransactionID,TransactionDate,CardType,BusinessDay,EntryStationID,ExitStationID,TicketType,TicketName,PassengerType,StationName,LineCode,Right_StationName,Right_LineCode,DateKey,DayType,DayNumberOfMonth,EnglishDayNameOfWeek,ThaiDayNameOfWeek,EnglishMonthName,ThaiMonthName,MonthNumberOfYear,CalendarQuater,CalendarSemester,CalendarYear
0,40,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
1,69,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
2,127,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
3,129,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017
4,333,2017-01-01,1,2017-01-01,22,21,Card,Adult-A,Adult,Sam Yan,BL,Hua Lamphong,BL,20170101,WE,1,Sunday,อาทิตย์,January,มกราคม,1,1,1,2017


In [75]:
transaction_df = get_transaction_df(jan_df)
transaction_df.head()

,TransactionDate,TransactionCounts
0,2017-01-25,37135
1,2017-01-26,36485
2,2017-01-13,36171
3,2017-01-24,36050
4,2017-01-20,35898


# Prep ข้อมูล เข้าและออกทั้งหมดของแต่ละสถานี

In [76]:
 entry_df = get_entry_total_station_by_date(jan_df , transaction_df , "StationName")
 entry_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Entry,Sam Yan,1328
1,2017-01-25,Entry,Si Lom,2865
2,2017-01-25,Entry,Lumphini,1448
3,2017-01-25,Entry,Khlong Toei,308
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400
...,...,...,...,...
553,2017-01-01,Entry,Phahon Yothin,352
554,2017-01-01,Entry,Chatuchak Park,359
555,2017-01-01,Entry,Kamphaeng Phet,233
556,2017-01-01,Entry,Bang Sue,186


In [77]:
exit_df = get_exit_total_station_by_date(jan_df , transaction_df , "Right_StationName")
exit_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Exit,Hua Lamphong,1338
1,2017-01-25,Exit,Sam Yan,1299
2,2017-01-25,Exit,Si Lom,2720
3,2017-01-25,Exit,Lumphini,1621
4,2017-01-25,Exit,Khlong Toei,289
...,...,...,...,...
553,2017-01-01,Exit,Lat Phrao,309
554,2017-01-01,Exit,Phahon Yothin,342
555,2017-01-01,Exit,Chatuchak Park,250
556,2017-01-01,Exit,Kamphaeng Phet,151


In [78]:
jan_transaction_df = entry_df.append(exit_df) 
jan_transaction_df

,TransactionDate,Type,Station,Count
0,2017-01-25,Entry,Sam Yan,1328
1,2017-01-25,Entry,Si Lom,2865
2,2017-01-25,Entry,Lumphini,1448
3,2017-01-25,Entry,Khlong Toei,308
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400
...,...,...,...,...
553,2017-01-01,Exit,Lat Phrao,309
554,2017-01-01,Exit,Phahon Yothin,342
555,2017-01-01,Exit,Chatuchak Park,250
556,2017-01-01,Exit,Kamphaeng Phet,151


In [79]:
jan_date_all_df = jan_transaction_df['TransactionDate']
jan_date_all_df

0      2017-01-25
1      2017-01-25
2      2017-01-25
3      2017-01-25
4      2017-01-25
          ...    
553    2017-01-01
554    2017-01-01
555    2017-01-01
556    2017-01-01
557    2017-01-01
Name: TransactionDate, Length: 1116, dtype: object

# Prep ข้อมูลประเภทของวัน(วันทำงาน , วันหยุด , วันนักขัตฤกษ์)

In [80]:
def get_day_type_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output
    # df = pd.DataFrame(output, columns =['TransactionDate','Type', 'Station','Count']) 
    # return df

In [81]:
day_type_list = get_day_type_list(jan_df , transaction_df , "DayType")

['WE', 'PH', 'WD']
[['2017-01-25', 0, 0, 37135], ['2017-01-26', 0, 0, 36485], ['2017-01-13', 0, 0, 36171], ['2017-01-24', 0, 0, 36050], ['2017-01-20', 0, 0, 35898], ['2017-01-11', 0, 0, 35831], ['2017-01-18', 0, 0, 35575], ['2017-01-10', 0, 0, 35343], ['2017-01-31', 0, 0, 35253], ['2017-01-19', 0, 0, 35208], ['2017-01-17', 0, 0, 35107], ['2017-01-09', 0, 0, 34666], ['2017-01-23', 0, 0, 34660], ['2017-01-06', 0, 0, 33946], ['2017-01-27', 0, 0, 33717], ['2017-01-16', 0, 0, 33200], ['2017-01-05', 0, 0, 33129], ['2017-01-30', 0, 0, 32814], ['2017-01-12', 0, 0, 31840], ['2017-01-04', 0, 0, 31479], ['2017-01-28', 16397, 0, 0], ['2017-01-21', 16184, 0, 0], ['2017-01-14', 16018, 0, 0], ['2017-01-07', 15156, 0, 0], ['2017-01-29', 13026, 0, 0], ['2017-01-03', 0, 12134, 0], ['2017-01-22', 12071, 0, 0], ['2017-01-15', 12068, 0, 0], ['2017-01-08', 11123, 0, 0], ['2017-01-02', 0, 7217, 0], ['2017-01-01', 5045, 0, 0]]


In [82]:
day_type_temp = pd.DataFrame(day_type_list, columns =['Date','WE','PH','WD'])
day_type_temp['DayType'] = ( (day_type_temp["WE"] != 0) * 1 ) + ( (day_type_temp["PH"] != 0) * 2 ) + ( (day_type_temp["WD"] != 0) * 3 )
day_type_temp.loc[day_type_temp['DayType'] == 1, 'DayType'] = "WE"
day_type_temp.loc[day_type_temp['DayType'] == 2, 'DayType'] = "PH"
day_type_temp.loc[day_type_temp['DayType'] == 3, 'DayType'] = "WD"
day_type_temp.drop('WE', axis=1, inplace=True)
day_type_temp.drop('PH', axis=1, inplace=True)
day_type_temp.drop('WD', axis=1, inplace=True)

In [83]:
jan_transaction_df = pd.merge(left=jan_transaction_df, right=day_type_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,Type,Station,Count,DayType
0,2017-01-25,Entry,Sam Yan,1328,WD
1,2017-01-25,Entry,Si Lom,2865,WD
2,2017-01-25,Entry,Lumphini,1448,WD
3,2017-01-25,Entry,Khlong Toei,308,WD
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400,WD
...,...,...,...,...,...
1111,2017-01-01,Exit,Lat Phrao,309,WE
1112,2017-01-01,Exit,Phahon Yothin,342,WE
1113,2017-01-01,Exit,Chatuchak Park,250,WE
1114,2017-01-01,Exit,Kamphaeng Phet,151,WE


# Prep ข้อมูลวันอะไรของสัปดาห์


In [85]:
def get_day_of_week_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      lst = []
      lst.append(date)
      for col_val in col_list:
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
      output.append(lst)
    print(output)
    return output

In [86]:
day_of_week_list = get_day_of_week_list(jan_df , transaction_df , "EnglishDayNameOfWeek")

['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
[['2017-01-25', 0, 0, 0, 37135, 0, 0, 0], ['2017-01-26', 0, 0, 0, 0, 36485, 0, 0], ['2017-01-13', 0, 0, 0, 0, 0, 36171, 0], ['2017-01-24', 0, 0, 36050, 0, 0, 0, 0], ['2017-01-20', 0, 0, 0, 0, 0, 35898, 0], ['2017-01-11', 0, 0, 0, 35831, 0, 0, 0], ['2017-01-18', 0, 0, 0, 35575, 0, 0, 0], ['2017-01-10', 0, 0, 35343, 0, 0, 0, 0], ['2017-01-31', 0, 0, 35253, 0, 0, 0, 0], ['2017-01-19', 0, 0, 0, 0, 35208, 0, 0], ['2017-01-17', 0, 0, 35107, 0, 0, 0, 0], ['2017-01-09', 0, 34666, 0, 0, 0, 0, 0], ['2017-01-23', 0, 34660, 0, 0, 0, 0, 0], ['2017-01-06', 0, 0, 0, 0, 0, 33946, 0], ['2017-01-27', 0, 0, 0, 0, 0, 33717, 0], ['2017-01-16', 0, 33200, 0, 0, 0, 0, 0], ['2017-01-05', 0, 0, 0, 0, 33129, 0, 0], ['2017-01-30', 0, 32814, 0, 0, 0, 0, 0], ['2017-01-12', 0, 0, 0, 0, 31840, 0, 0], ['2017-01-04', 0, 0, 0, 31479, 0, 0, 0], ['2017-01-28', 0, 0, 0, 0, 0, 0, 16397], ['2017-01-21', 0, 0, 0, 0, 0, 0, 16184], ['2017-01-14', 0, 

In [88]:
day_of_week_unique = jan_df['EnglishDayNameOfWeek'].unique().tolist()
day_of_week_unique.insert(0,"Date")

['Date', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']


In [95]:
day_of_week_temp = pd.DataFrame(day_of_week_list, columns =day_of_week_unique)
day_of_week_temp['DateOfWeek'] = ( (day_of_week_temp["Sunday"] != 0) * 1 ) + ( (day_of_week_temp["Monday"] != 0) * 2 ) + ( (day_of_week_temp["Tuesday"] != 0) * 3 )+ ( (day_of_week_temp["Wednesday"] != 0) * 4 )+ ( (day_of_week_temp["Thursday"] != 0) * 5 )+ ( (day_of_week_temp["Friday"] != 0) * 6 )+ ( (day_of_week_temp["Saturday"] != 0) * 7 )
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 1, 'DateOfWeek'] = "Sunday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 2, 'DateOfWeek'] = "Monday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 3, 'DateOfWeek'] = "Tuesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 4, 'DateOfWeek'] = "Wednesday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 5, 'DateOfWeek'] = "Thursday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 6, 'DateOfWeek'] = "Friday"
day_of_week_temp.loc[day_of_week_temp['DateOfWeek'] == 7, 'DateOfWeek'] = "Saturday"
for col in jan_df['EnglishDayNameOfWeek'].unique().tolist():
  day_of_week_temp.drop(col, axis=1, inplace=True)

In [96]:
jan_transaction_df = pd.merge(left=jan_transaction_df, right=day_of_week_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,Type,Station,Count,DayType,DateOfWeek
0,2017-01-25,Entry,Sam Yan,1328,WD,Wednesday
1,2017-01-25,Entry,Si Lom,2865,WD,Wednesday
2,2017-01-25,Entry,Lumphini,1448,WD,Wednesday
3,2017-01-25,Entry,Khlong Toei,308,WD,Wednesday
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400,WD,Wednesday
...,...,...,...,...,...,...
1111,2017-01-01,Exit,Lat Phrao,309,WE,Sunday
1112,2017-01-01,Exit,Phahon Yothin,342,WE,Sunday
1113,2017-01-01,Exit,Chatuchak Park,250,WE,Sunday
1114,2017-01-01,Exit,Kamphaeng Phet,151,WE,Sunday


# Prep ข้อมูล วันที่เท่าไหร่ของเดือน และ เป็นช่วงไหนของเดือน (ต้นเดือน , กลางเดือน , สิ้นเดือน)

In [101]:
df_day_number_of_month = pd.DataFrame(columns=['DayNumberOfMonth','MonthPeriod','MonthNumberOfYear','CalendarQuater'])
df_day_number_of_month["DayNumberOfMonth"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthPeriod"] =  jan_transaction_df['TransactionDate'].str.slice(8, 10).astype('int')
df_day_number_of_month["MonthNumberOfYear"] =  jan_transaction_df['TransactionDate'].str.slice(5, 7).astype('int')

In [102]:
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') <= 5, 'MonthPeriod'] = 1
df_day_number_of_month.loc[( (df_day_number_of_month['MonthPeriod'].astype('int') > 5) & (df_day_number_of_month['MonthPeriod'].astype('int') < 25) ), 'MonthPeriod'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'].astype('int') >= 25, 'MonthPeriod'] = 3

df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 1, 'MonthPeriod'] = "Early"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 2, 'MonthPeriod'] = "Mid"
df_day_number_of_month.loc[df_day_number_of_month['MonthPeriod'] == 3, 'MonthPeriod'] = "Late"

In [103]:
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 1, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 2, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 3, 'CalendarQuater'] = 1
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 4, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 5, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 6, 'CalendarQuater'] = 2
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 7, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 8, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 9, 'CalendarQuater'] = 3
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 10, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 11, 'CalendarQuater'] = 4
df_day_number_of_month.loc[df_day_number_of_month['MonthNumberOfYear'].astype('int') == 12, 'CalendarQuater'] = 4

In [105]:
jan_transaction_df = jan_transaction_df.join(df_day_number_of_month)
jan_transaction_df

,TransactionDate,Type,Station,Count,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater
0,2017-01-25,Entry,Sam Yan,1328,WD,Wednesday,25,Late,1,1
1,2017-01-25,Entry,Si Lom,2865,WD,Wednesday,25,Late,1,1
2,2017-01-25,Entry,Lumphini,1448,WD,Wednesday,25,Late,1,1
3,2017-01-25,Entry,Khlong Toei,308,WD,Wednesday,25,Late,1,1
4,2017-01-25,Entry,Queen Sirikit National Convention Centre,1400,WD,Wednesday,25,Late,1,1
...,...,...,...,...,...,...,...,...,...,...
1111,2017-01-01,Exit,Lat Phrao,309,WE,Sunday,1,Early,1,1
1112,2017-01-01,Exit,Phahon Yothin,342,WE,Sunday,1,Early,1,1
1113,2017-01-01,Exit,Chatuchak Park,250,WE,Sunday,1,Early,1,1
1114,2017-01-01,Exit,Kamphaeng Phet,151,WE,Sunday,1,Early,1,1


# Prep ข้อมูล ประเภทผู้โดยสาร

In [127]:
def get_passenger_type_list(month_df , month_transaction_df , col_name):
    date_list = month_transaction_df['TransactionDate'].tolist()
    col_list = month_df[col_name].unique().tolist()
    print(col_list)
    output = []
    for date in date_list:
      for col_val in col_list:
        lst = []
        lst.append(date)
        lst.append(col_val)
        subset_df = month_df[ (month_df[col_name]==col_val) & (month_df["TransactionDate"]==date) ].count()
        column_count = subset_df.value_counts()
        lst.append(column_count.index[0])
        output.append(lst)
        # print("{} : {} : {}".format(date,col_val,column_count.index[0]) )
      # print("-----")
    print(output)
    return output

In [128]:
passenger_type_list = get_passenger_type_list(jan_df , transaction_df , "PassengerType")

['Adult', 'Senior', 'Student', 'Child']
[['2017-01-25', 'Adult', 30511], ['2017-01-25', 'Senior', 1823], ['2017-01-25', 'Student', 4719], ['2017-01-25', 'Child', 82], ['2017-01-26', 'Adult', 29929], ['2017-01-26', 'Senior', 1752], ['2017-01-26', 'Student', 4736], ['2017-01-26', 'Child', 68], ['2017-01-13', 'Adult', 29937], ['2017-01-13', 'Senior', 1640], ['2017-01-13', 'Student', 4519], ['2017-01-13', 'Child', 75], ['2017-01-24', 'Adult', 29784], ['2017-01-24', 'Senior', 1617], ['2017-01-24', 'Student', 4585], ['2017-01-24', 'Child', 64], ['2017-01-20', 'Adult', 29861], ['2017-01-20', 'Senior', 1646], ['2017-01-20', 'Student', 4342], ['2017-01-20', 'Child', 49], ['2017-01-11', 'Adult', 29852], ['2017-01-11', 'Senior', 1602], ['2017-01-11', 'Student', 4304], ['2017-01-11', 'Child', 73], ['2017-01-18', 'Adult', 29704], ['2017-01-18', 'Senior', 1628], ['2017-01-18', 'Student', 4167], ['2017-01-18', 'Child', 76], ['2017-01-10', 'Adult', 29534], ['2017-01-10', 'Senior', 1457], ['2017-01-10'

In [129]:
passenger_type_temp = pd.DataFrame(passenger_type_list, columns = ['Date','PassengerType','PassengerNumber'])
passenger_type_temp

,Date,PassengerType,PassengerNumber
0,2017-01-25,Adult,30511
1,2017-01-25,Senior,1823
2,2017-01-25,Student,4719
3,2017-01-25,Child,82
4,2017-01-26,Adult,29929
...,...,...,...
119,2017-01-02,Child,17
120,2017-01-01,Adult,4112
121,2017-01-01,Senior,472
122,2017-01-01,Student,455


In [130]:
jan_transaction_df = pd.merge(left=jan_transaction_df, right=passenger_type_temp, left_on='TransactionDate', right_on='Date')
jan_transaction_df.drop('Date', axis=1, inplace=True)
jan_transaction_df

,TransactionDate,Type,Station,Count,DayType,DateOfWeek,DayNumberOfMonth,MonthPeriod,MonthNumberOfYear,CalendarQuater,PassengerType,PassengerNumber
0,2017-01-25,Entry,Sam Yan,1328,WD,Wednesday,25,Late,1,1,Adult,30511
1,2017-01-25,Entry,Sam Yan,1328,WD,Wednesday,25,Late,1,1,Senior,1823
2,2017-01-25,Entry,Sam Yan,1328,WD,Wednesday,25,Late,1,1,Student,4719
3,2017-01-25,Entry,Sam Yan,1328,WD,Wednesday,25,Late,1,1,Child,82
4,2017-01-25,Entry,Si Lom,2865,WD,Wednesday,25,Late,1,1,Adult,30511
...,...,...,...,...,...,...,...,...,...,...,...,...
4459,2017-01-01,Exit,Kamphaeng Phet,151,WE,Sunday,1,Early,1,1,Child,6
4460,2017-01-01,Exit,Bang Sue,240,WE,Sunday,1,Early,1,1,Adult,4112
4461,2017-01-01,Exit,Bang Sue,240,WE,Sunday,1,Early,1,1,Senior,472
4462,2017-01-01,Exit,Bang Sue,240,WE,Sunday,1,Early,1,1,Student,455


# Prep ข้อมูล Events

In [114]:
events_df = pd.read_csv("gdrive/My Drive/NIDA/Events.csv")
events_df.drop('EventsId', axis=1, inplace=True)

In [115]:
events_df.shape

(176, 6)

In [116]:
events_df.head()

,EvenstName,EventsLocation,EventsType,EventsDateStart,EventsDateEnd,Interest
0,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.1,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2560-01-08,2560-01-08,2241
1,19th Bangkok International Symposium on HIV Me...,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,2560-01-18,2560-01-20,1565
2,IDP AUSportunity,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),การศึกษา,2560-01-21,2560-01-22,1229
3,WIS FORUM 2017,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),สัมมนา,2560-01-24,2560-01-26,1685
4,LEGAL EXECUTION DEPARTMENT AUCTION 2017 Ep.2,QSNCC (ศูนย์ประชุมฯ สิริกิติ์),กฏหมาย,2560-01-29,2560-01-29,1203


In [117]:
events_df.EventsLocation.value_counts()

QSNCC (ศูนย์ประชุมฯ สิริกิติ์)    111
เซ็นทรัล ลาดพร้าว                  42
เซ็นทรัล พระราม9                   12
TERMINAL 21                        11
Name: EventsLocation, dtype: int64

In [118]:
events_df.EventsType.value_counts()

ทั่วไป             32
สัมมนา             21
แฟชั่น             16
ที่อยู่อาศัย       16
กีฬา               11
ธุรกิจ             11
อาหาร              10
ท่องเที่ยว          9
สินค้า IT           8
การศึกษา            5
สัตว์เลี้ยง         5
เกษตร               4
ครอบครัว            4
การเงิน             3
หนังสือ             3
เครื่องใช้ไฟฟ้า     3
ความบันเทิง         3
อาชีพ               2
สุขภาพ              2
กฏหมาย              2
Design              2
พาหนะ               2
Beauty              1
งานแต่ง             1
Name: EventsType, dtype: int64

# Export Data 

In [131]:
jan_transaction_df.to_excel (r'Jan_Summary.xlsx', index = False, header=True)